# モデルの作成

In [1]:
!python main.py

Using TensorFlow backend.





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/Users/matsusesatoshi/DIVE_2/Unet/model.py:55: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/1
Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.


2021-09-03 22:51:06.260237: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-09-03 22:51:06.315805: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fdee2954770 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-09-03 22:51:06.315871: I tensorflow/compiler/xla/service/service.cc:176]   StreamE

# 【問題1】学習・推定
以下のKeras実装を使用して学習・推定を行ってください。


zhixuhao/unet: unet for image segmentation


《GPU環境での学習》


大規模なデータセット、大きなモデルになるため、GPUを使用する必要があります。


《新たなデータセットの適用》


公開されている実装で用意されたものとは異なるデータセットを入力するための準備が必要です。

In [2]:
from model import *
from data import *

Using TensorFlow backend.


In [3]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(2,'data/membrane/train','image','label',data_gen_args,save_to_dir = None)
model = unet()
model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(myGene,steps_per_epoch=2000,epochs=1,callbacks=[model_checkpoint])






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/Users/matsusesatoshi/DIVE_2/Unet/model.py:55: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)





Epoch 1/1
Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.





2000/2000 [==============================] - 7009s 4s/step - loss: 0.6675 - acc: 0.7812

Epoch 00001: loss improved from inf to 0.66745, saving model to unet_membrane.hdf5


In [4]:
testGene = testGenerator("data/membrane/test")
model = unet()
model.load_weights("unet_membrane.hdf5")
results = model.predict_generator(testGene,30,verbose=1)
saveResult("data/membrane/test",results)

30/30 [==============================] - 13s 434ms/step


/Users/matsusesatoshi/DIVE_2/Unet/data.py:124: UserWarning: data/membrane/test/0_predict.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/Users/matsusesatoshi/DIVE_2/Unet/data.py:124: UserWarning: data/membrane/test/1_predict.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/Users/matsusesatoshi/DIVE_2/Unet/data.py:124: UserWarning: data/membrane/test/2_predict.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/Users/matsusesatoshi/DIVE_2/Unet/data.py:124: UserWarning: data/membrane/test/3_predict.png is a low contrast i

# 【問題2】コードリーディング
論文[1]に目を通した上で、上記実装のコードリーディングを行ってください。

ランタイムの限界

CNNの畳み込み層、プーリング層を重ねて行い、画像の特徴量をch方向に重ねる事でエッジ等の重要な情報だけを抜き出す。  
通常であれば、そのあと全結合層にいれて分類問題を解くためのNNとするが、ここでは畳み込み層のまま出力する。  
ここから、逆畳み込み層を導入して画像を元のサイズまで復元していく事を目的としている。  
その際、同サイズの元の画像を入力させる事で推定領域を細かなピクセル単位で出力することが可能となる

![Imgur](https://i.imgur.com/epmktcDh.png)